In [1]:
# Imports
import torch
from torch.distributions import Normal,Gamma,Bernoulli
from dask_jobqueue import SLURMCluster
from distributed import Client
import CSuite_Cocycles

In [2]:
# Experiment args set up
ntrial = 20
datasets = ["Fork","Simpson","NonlinG"]
dists = [Normal(0,1),Bernoulli(0.5),Gamma(1,1)]
transforms = [lambda x : x, lambda x : 2*x - 1, lambda x : 1 / x - 1]
num_change = [0,4,4]

# Model args set up
cocycle_losses = ["CMMD_U","CMMD_V","LS"]
batch_size = 64
scheduler = False
decay = 0

In [3]:
cluster = SLURMCluster(
    n_workers=0,
    memory="20GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":10095",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="48:0:0",
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [4]:
futures = []
for seed in range(ntrial):
    for d in range(len(datasets)):
        for j in range(len(dists)):
            for m in range(len(cocycle_losses)):
                f = client.submit(CSuite_Cocycles.run_experiment,seed,datasets[d],
                                                              dists[j],
                                                              transforms[j],
                                                              num_change[j],
                                                              cocycle_losses[m],
                                                              batch_size,
                                                              scheduler,
                                                              decay)
                futures.append(f)

In [5]:
results = client.gather(futures)

In [6]:
client.close()
cluster.close()

2025-03-20 14:59:44,411 - distributed.deploy.adaptive_core - INFO - Adaptive stop


In [7]:
torch.save(f = "Csuite_Cocycles_trials={0}_scheduler={1}_decay={2}_CV.pt".format(ntrial,scheduler,decay), obj = results)